In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.core import Dense,Activation,Dropout,Flatten
from keras.datasets import cifar10
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
import dlt
import os
from PIL import Image
import pandas as pd

import random
import shutil


#データセットを読み込んでテンソルに変換

#画像名リスト作成
test_filename = os.listdir('./500test')           #ここで指定したディレクトリをまとめてnp配列に変換できる。
true_filename = os.listdir('./10000harf')



#random.shuffle(test_filename)
#random.shuffle(true_filename)

#dcgan_filename.sort()
#print(dcgan_filename)

if ".ipynb_checkpoints" in test_filename:
    test_filename.remove(".ipynb_checkpoints")     #いらないものを消す。

if ".ipynb_checkpoints" in true_filename:
    true_filename.remove(".ipynb_checkpoints")
    


#print(len(true_filename))


#クラスリスト作成
#ex:classmodeはint型で入る。
def makeusable(classmode,filename):
    classname = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
    list1 = []
    i=0
    
    if classmode == 0:
        for name in filename:
            if classname[0] in name:
                list1.append([1])
            else:
                list1.append([0])
                
    elif classmode == 1: 
        for name in filename:
            if classname[1] in name:
                list1.append([1])
            else:
                list1.append([0])
                
    elif classmode == 2: 
        for name in filename:
            if classname[2] in name:
                list1.append([1])
            else:
                list1.append([0])
                
        
    elif classmode == 3: 
        for name in filename:
            if classname[3] in name:
                list1.append([1])
            else:
                list1.append([0])
                
    elif classmode == 4: 
        for name in filename:
            if classname[4] in name:
                list1.append([1])
            else:
                list1.append([0])
            
    elif classmode == 5: 
        for name in filename:
            if classname[5] in name:
                list1.append([1])
            else:
                list1.append([0])
                
    elif classmode == 6: 
        for name in filename:
            if classname[6] in name:
                list1.append([1])
            else:
                list1.append([0])
                
    elif classmode == 7: 
        for name in filename:
            if classname[7] in name:
                list1.append([1])
            else:
                list1.append([0])
                         
    elif classmode == 8: 
        for name in filename:
            if classname[8] in name:
                list1.append([1])
            else:
                list1.append([0])
                
    else:
        for name in filename:
            if classname[9] in name:
                list1.append([1])
            else:
                list1.append([0])
                
            #print(str(i)+":"+str(name))
            i+=1
                
                
    #list型をnp.arrayに変換
    classlist = np.array(list1)
    return classlist

#
#classmodeの設定！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
#ここで対象クラスを指定
#

classname = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

#統計用リスト
kklist = []

for classmode in range(len(classname)):

    fname = './20000dcpic_c/{}'.format(classname[classmode])
    dcgan_filename = os.listdir(fname)

    if ".ipynb_checkpoints" in dcgan_filename:
        dcgan_filename.remove(".ipynb_checkpoints")  


    #画像ファイルの相対パスを取得
    test_filepass = []
    for name in test_filename:
        test_filepass.append("./500test/"+name)

    #print(filepass)

    true_filepass = []
    for name in true_filename:
        true_filepass.append("./10000harf/"+name)

    dcgan_filepass = []
    for name in dcgan_filename:
        dcgan_filepass.append(fname+"/"+name)


    test_label = makeusable(classmode,test_filename)
    true_label = makeusable(classmode,true_filename)

    dcgan_label = makeusable(classmode,dcgan_filename)



    #test画像
    #png画像をndarrayに変換。→一旦listに直して、imlistに追加
    test_imlist = []
    for im in test_filepass:
        pic = np.array(Image.open(im))
        piclist = pic.tolist()
        test_imlist.append(piclist)

    #imlist（list型）をimarray（ndarray型）に変換
    test_imarray = np.array(test_imlist)


    #true画像
    #png画像をndarrayに変換。→一旦listに直して、imlistに追加
    true_imlist = []
    for im in true_filepass:
        pic = np.array(Image.open(im))
        piclist = pic.tolist()
        true_imlist.append(piclist)

    #imlist（list型）をimarray（ndarray型）に変換
    true_imarray = np.array(true_imlist)

    #print(true_filepass)
    #print(len(true_filepass),len(true_label))
    #print(len(x))

    #dcgan画像
    #png画像をndarrayに変換。→一旦listに直して、imlistに追加
    dcgan_imlist = []
    for im in dcgan_filepass:
        pic = np.array(Image.open(im))
        piclist = pic.tolist()
        dcgan_imlist.append(piclist)

    #imlist（list型）をimarray（ndarray型）に変換
    dcgan_imarray = np.array(dcgan_imlist)


    #ここで2つのパターンを用意する。
    #パターン１：学習段階で、usableを５００枚（１クラス分）＋unusableを４５００枚（他の９クラス全部）
    #パターン２：学習段階で、usableを５００枚（１クラス分）＋unusableを５００枚（他の９クラスからランダムに５００枚選出）←今これ
    #
    #これらでそれぞれ学習させ、精度の良い方を使用。


    #cifar10のデータセットを設定。テスト画像も変えてね。
    (x_train,y_train)=(true_imarray,true_label)
    (x_test,y_test)=(test_imarray,test_label)


    #画像を0-1の範囲で正規化
    x_train=x_train.astype('float32')/255.0
    x_test=x_test.astype('float32')/255.0

    #正解ラベルをOne-Hot表現に変換
    y_train=np_utils.to_categorical(y_train,2)
    y_test=np_utils.to_categorical(y_test,2)

    #統計用リスト
    picnum = []
    fcount = []
    acclist = []
    prelabellist = []

    for n in range(2000):
        picnum.append(n)
        fcount.append(0)

    #１０回学習
    for num in range(10):
        #モデルを構築
        model=Sequential()

        model.add(Conv2D(64,(3,3),padding='same',input_shape=(32,32,3)))
        model.add(Activation('relu'))
        model.add(Conv2D(64,(3,3),padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        #model.add(Dropout(0.25))

        model.add(Conv2D(128,(3,3),padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(128,(3,3),padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        #model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(2,activation='softmax'))

        model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])



        history=model.fit(x_train,y_train,batch_size=128,epochs=20,verbose=1,validation_split=0.1)

        #モデルと重みを保存
        json_string=model.to_json()
        open('cifar10_cnn.json',"w").write(json_string)
        savefilename = '{}_cnn.h5'.format(classname[classmode])
        model.save_weights(savefilename)

        #モデルの表示/
        #model.summary()




        #評価
        score=model.evaluate(x_test,y_test,verbose=0)
        print('Test loss:',score[0])
        print('Test accuracy:',score[1])


        #ファイル選別のためのアドレスを生成

        acc_count = []
        usable = []
        unusable = []

        x_usable = dcgan_imarray
        y_usable = dcgan_label

        y_predict = model.predict(x_usable, batch_size=32)
        predict_classes = np.argmax(y_predict,1)
        true_classes = np.argmax(y_usable,1)

        for i in range(len(predict_classes)):
            if predict_classes[i] == [1]:
                acc_count.append(1)    
                usable.append(i)


            else:
                acc_count.append(0)
                unusable.append(i)


        acc = np.average(acc_count)

        #acc_count：usableなら１,unusableなら０となっている、予測されたラベルのリスト
        prelabellist.append(acc_count)


        for num in usable:
            fcount[num] += 1



        print(acc)
        acclist.append(acc)
        #print(usable)
        #print(unusable)

    data = {
        "通し番号" : picnum,
        "カウント" : fcount
    }

    df = pd.DataFrame(data, columns=["通し番号", "カウント"])

    predicts = {
        "通し番号" : picnum,
        "１回目" : prelabellist[0],
        "２回目" : prelabellist[1],
        "３回目" : prelabellist[2],
        "４回目" : prelabellist[3],
        "５回目" : prelabellist[4],
        "６回目" : prelabellist[5],
        "７回目" : prelabellist[6],
        "８回目" : prelabellist[7],
        "９回目" : prelabellist[8],
        "１０回目" : prelabellist[9]
    }

    pds = pd.DataFrame(predicts, columns=["通し番号","１回目","２回目","３回目","４回目","５回目","６回目","７回目","８回目","９回目","１０回目"])

    print(df)
    print(acclist)
    df.to_csv("ave10.csv")
    pds.to_csv("plabellist.csv")
    
    dfs = df.sort_values("カウント",ascending=False)

    dfname = "ave10_c{}_sort.csv".format(classmode)
    dfs.to_csv(dfname)

    #ファイルの選別
    #画像をスコアごとリストにまとめる。後、一つのリストに結合。
    slist = []
    sortbyscore = []
    for sc in range(11):
        exec("cnt{} = []".format(sc))
        exec("slist.append(cnt{})".format(sc))

    for sc in range(11):
        con = "カウント == {}".format(sc)
        df0 = dfs.query(con)
        slist[sc] = list(df0["通し番号"])

    print(len(slist))

    for sc in range(len(slist)):
        sortbyscore = sortbyscore + slist[sc]


    print("\n\n\n\n")
    print(sortbyscore)
    #本当に使える画像（Tusable）と本当に使えなさそうな画像（Tunusable）に分ける。

    Tusable = sortbyscore[1500:2000]
    Tunusable = sortbyscore[0:500]

    print(len(Tusable))
    print(len(Tunusable))

    #ディレクトリに入れる
    for i in range(len(Tusable)):
        file = fname+"/"+str(dcgan_filename[Tusable[i]])
        target = "./usable/{}".format(str(dcgan_filename[Tusable[i]]))
        shutil.copyfile(file, target)


    for i in range(len(Tunusable)):
        file = fname+"/"+str(dcgan_filename[Tunusable[i]])
        target = "./unusable/{}".format(str(dcgan_filename[Tunusable[i]]))
        shutil.copyfile(file, target)
        
    klist = []
    for i in range(11):
        txt = "カウント == {}".format(i)
        dfn = dfs.query(txt)
        klist.append(len(dfn))
        
    kklist.append(klist)
    
#pd.Series(klist)

    

2022-11-29 16:02:10.556261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 16:02:10.623619: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-29 16:02:10.642174: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-29 16:02:10.961536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Epoch 1/20


2022-11-29 16:02:18.777785: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2022-11-29 16:02:19.224176: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


36/36 [==============================] - 2s 19ms/step - loss: 0.3544 - accuracy: 0.8904 - val_loss: 0.2928 - val_accuracy: 0.9120
Epoch 2/20
36/36 [==============================] - 0s 11ms/step - loss: 0.2631 - accuracy: 0.8987 - val_loss: 0.2861 - val_accuracy: 0.9120
Epoch 3/20
36/36 [==============================] - 0s 11ms/step - loss: 0.2590 - accuracy: 0.9004 - val_loss: 0.2147 - val_accuracy: 0.9200
Epoch 4/20
36/36 [==============================] - 0s 11ms/step - loss: 0.2405 - accuracy: 0.9042 - val_loss: 0.2203 - val_accuracy: 0.9260
Epoch 5/20
36/36 [==============================] - 0s 11ms/step - loss: 0.2360 - accuracy: 0.9060 - val_loss: 0.2397 - val_accuracy: 0.9260
Epoch 6/20
36/36 [==============================] - 0s 11ms/step - loss: 0.2350 - accuracy: 0.9062 - val_loss: 0.2001 - val_accuracy: 0.9360
Epoch 7/20
36/36 [==============================] - 0s 11ms/step - loss: 0.2132 - accuracy: 0.9082 - val_loss: 0.2086 - val_accuracy: 0.9360
Epoch 8/20
36/36 [======

In [2]:
print(np.average(acclist))
print(np.var(acclist))

0.58125
0.0310497625


In [3]:
"""
#ファイル選別のためのアドレスを生成

acc_count = []
usable = []
unusable = []

x_usable = dcgan_imarray
y_usable = dcgan_label

y_predict = model.predict(x_usable, batch_size=32)
predict_classes = np.argmax(y_predict,1)
true_classes = np.argmax(y_usable,1)

for i in range(len(predict_classes)):
    if predict_classes[i] == [1]:
        acc_count.append(1)    
        usable.append(i)
        
        
    else:
        acc_count.append(0)
        unusable.append(i)


acc = np.average(acc_count)


for num in usable:
    fcount[num] += 1



print(acc)
#print(usable)
#print(unusable)
"""

'\n#ファイル選別のためのアドレスを生成\n\nacc_count = []\nusable = []\nunusable = []\n\nx_usable = dcgan_imarray\ny_usable = dcgan_label\n\ny_predict = model.predict(x_usable, batch_size=32)\npredict_classes = np.argmax(y_predict,1)\ntrue_classes = np.argmax(y_usable,1)\n\nfor i in range(len(predict_classes)):\n    if predict_classes[i] == [1]:\n        acc_count.append(1)    \n        usable.append(i)\n        \n        \n    else:\n        acc_count.append(0)\n        unusable.append(i)\n\n\nacc = np.average(acc_count)\n\n\nfor num in usable:\n    fcount[num] += 1\n\n\n\nprint(acc)\n#print(usable)\n#print(unusable)\n'

In [4]:
#ファイルの選別（usable/unusable）

import shutil
"""
for i in range(len(usable)):
    file = "./test_model2/"+str(dcgan_filename[usable[i]])
    target = "test_model2_us/{}".format(str(dcgan_filename[usable[i]]))
    shutil.copyfile(file, target)


for j in range(len(unusable)):
    file = "./test_model2/"+str(dcgan_filename[unusable[j]])
    target = "test_model2_un/{}".format(str(dcgan_filename[unusable[j]]))
    shutil.copyfile(file, target)


print("finish")

"""

'\nfor i in range(len(usable)):\n    file = "./test_model2/"+str(dcgan_filename[usable[i]])\n    target = "test_model2_us/{}".format(str(dcgan_filename[usable[i]]))\n    shutil.copyfile(file, target)\n\n\nfor j in range(len(unusable)):\n    file = "./test_model2/"+str(dcgan_filename[unusable[j]])\n    target = "test_model2_un/{}".format(str(dcgan_filename[unusable[j]]))\n    shutil.copyfile(file, target)\n\n\nprint("finish")\n\n'

In [5]:
print(len(x_train))

5000
